In [1]:
import os
import pathlib
import time
import datetime

from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import glob
import json
import random
import csv
import concurrent.futures
from multiprocessing import freeze_support

from keras import layers
from keras import models
from keras import mixed_precision

seed = 6969
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

replays_dir = pathlib.Path("replays")

In [2]:
pre_segment_size = 3
post_segment_size = 3
prediction_size = 3
segment_size = pre_segment_size + post_segment_size + prediction_size
batch_size = 256

In [9]:

def get_leaderboard_replays():
  leaderboard_ids = np.array(tf.io.gfile.listdir(str(replays_dir)))
  random.shuffle(leaderboard_ids)
  val_leaderboard_ids = leaderboard_ids[:int(len(leaderboard_ids)*0.2)]

  train_data = []
  val_data = []
  for leaderboard_id in leaderboard_ids:
    replay_files = glob.glob(f'{replays_dir}/{leaderboard_id}/*.npy')

    if leaderboard_id in val_leaderboard_ids:
      val_data.append((leaderboard_id, replay_files))
    else:
      train_data.append((leaderboard_id, replay_files))

  return train_data, val_data

def get_replay_notes(replay):
  notes = []
  
  prev_zero_note_time = 0
  prev_one_note_time = 0
  
  for note_info, score, note_time in sorted(replay, key=lambda item: item[2]):
    type = note_info[-2]
    
    # NOTE: 0-100 score range is rare and often happens for tracking problems that are not important here
    # would be good to replace this with acc component only and potentially learn all both acc and swing angles
    # but need different format replay files for that
    # score = max(0, score - 100)
    
    delta_to_zero = note_time - prev_zero_note_time
    delta_to_one = note_time - prev_one_note_time
    
    if type == "0":
      prev_zero_note_time = note_time
      note = preprocess_note(score, delta_to_zero, delta_to_one, note_info)
      notes.append(note)
    if type == "1":
      prev_one_note_time = note_time
      note = preprocess_note(score, delta_to_one, delta_to_zero, note_info)
      notes.append(note)
  
  return notes

def preprocess_note(score, delta, delta_other, note_info):
  # NOTE: timing increases difficulty not linearly and caps out at ~2 seconds
  # no idea if such parameters can be learned by neural networks without adding scaling like I did right here
  delta = max(0, 1.5 - delta)
  delta_other = max(0, 1.5 - delta_other)

  col_number = int(note_info[1])
  row_number = int(note_info[2])
  direction_number = int(note_info[4])
  color = int(note_info[3])
  color_arr = [0, 0]
  color_arr[color] = 1

  col = [0] * 4
  row = [0] * 3
  row_col = [0] * 4 * 3
  direction = [0] * 10
  col[col_number] = 1
  row[row_number] = 1
  row_col[col_number * 3 + row_number] = 1
  direction[direction_number] = 1

  response = []
  
  response.extend(row_col)
  response.extend(direction)
  response.extend(color_arr)
  response.extend([
    delta,
    delta_other,
    score
  ])
  
  return response

def create_segments(notes):
  if len(notes) < segment_size:
    return [], [], [], []
  
  pre_segments = []
  segments = []
  post_segments = []
  scores = []
  for i in range(len(notes) - segment_size):
    if i % prediction_size != 0:
      continue
    
    pre_slice = notes[i:i+pre_segment_size]
    slice = notes[i+pre_segment_size:i+pre_segment_size+prediction_size]
    post_slice = notes[i+pre_segment_size+prediction_size:i+segment_size]

    # NOTE: using relative score can be good to find relative difficulty of the notes more fairly
    # because good players will always get higher acc and worse players will do badly even on easy patterns

    pre_segment = [np.array(note[:-1]) for note in pre_slice]
    segment = [np.array(note[:-1]) for note in slice]
    post_segment = [np.array(note[:-1]) for note in post_slice]

    score = sum([note[-1]/15 for note in slice])/len(slice)

    pre_segments.append(pre_segment)
    segments.append(segment)
    post_segments.append(post_segment)
    scores.append(score)
    
  return pre_segments, segments, post_segments, scores

def to_id(value):
  if int(value) < 30000:
    return str(30000 + int(value))
  return str(int(value))

def preprocess_leaderboard_replays(leaderboard_replays, print_progress=False):
  asd = {}
  replays = []
  
  for leaderboard_replay in leaderboard_replays:
    arr = np.load(leaderboard_replay)
    sort_indices = np.argsort(arr[:, 2])

    # Sort the entire array column-wise based on these indices
    sorted_arr = arr[sort_indices]
    replays.append(sorted_arr)
  
  for replay in replays:
    if len(asd.values()) == 0:
      for values in replay:
        asd[to_id(values[0]) + str(values[2])] = [to_id(values[0]), [values[1]], values[2], 1]
    else:
      for values in replay:
        key = to_id(values[0]) + str(values[2])
        if key in asd:
          asd[key][1].append(values[1])
          asd[key][2] += values[2]
          asd[key][3] += 1
  
  asd2 = []
  for values in asd.values():
    if values[3] < 5 or len(values[0]) > 5:
      return [], [], [], []

    acc_list = values[1]
    acc = (sum(acc_list) * 1000) / len(acc_list) if len(acc_list) > 0 else 0
    asd2.append([values[0], acc, values[2] / values[3]])
  
  notes = get_replay_notes(asd2)
  return create_segments(notes)


def generate_data(leaderboards_replays):
  pre_segments = []
  segments = []
  post_segments = []
  scores = []
  
  for (leaderboard_id, leaderboard_replays) in leaderboards_replays:
    pre_segment, segment, post_segment, score = preprocess_leaderboard_replays(leaderboard_replays)
    if len(pre_segment) == 0: 
      continue

    pre_segments.extend(pre_segment)
    segments.extend(segment)
    post_segments.extend(post_segment)
    scores.extend(score)
  
  return [np.array(pre_segments), np.array(segments), np.array(post_segments)], np.array(scores)

def clamp(num, lower, higher):
  return min([max([num, lower]), higher])

train_data, val_data = get_leaderboard_replays()
test_data = val_data

train_x, train_y = generate_data(train_data)
val_x, val_y = generate_data(val_data)
note_size = 26

pre_input = keras.Input(shape=(pre_segment_size, note_size), dtype="float32")
pre_layer = layers.Flatten()(pre_input)
pre_layer = layers.Dense(32, activation="relu")(pre_layer)
pre_layer = layers.Dense(32, activation="relu")(pre_layer)
input = keras.Input(shape=(prediction_size, note_size), dtype="float32")
layer = layers.Flatten()(input)
layer = layers.Dense(32, activation="relu")(layer)
layer = layers.Dense(32, activation="relu")(layer)
post_input = keras.Input(shape=(post_segment_size, note_size), dtype="float32")
post_layer = layers.Flatten()(post_input)
post_layer = layers.Dense(32, activation="relu")(post_layer)
post_layer = layers.Dense(32, activation="relu")(post_layer)
  
inputs = [pre_input, input, post_input]
layers2 = [pre_layer, layer, post_layer]
l = layers.Concatenate()(layers2)
l = layers.Flatten()(l)
l = layers.Dense(64, activation="relu")(l)
l = layers.Dense(64, activation="relu")(l)
out = layers.Dense(1, activation="linear")(l)
model = models.Model(inputs=inputs, outputs = out)
model.summary()

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss=tf.keras.losses.MeanAbsoluteError(reduction="sum_over_batch_size"),
    metrics=['mae', 'mse'],
)

tot = 0
for score in train_y:
    tot += score
avg = tot/len(train_y)
totdiff = 0

for score in train_y:
    totdiff += max(score - avg, avg - score)
avgdiff = totdiff/len(train_y)

print(f"Average value: {avg}")
print(f"Average diff: {avgdiff}")

totdiff = 0
for score in val_y:
    totdiff += max(score - avg, avg - score)
avgdiff = totdiff/len(val_y)
print(f"Average diff: {avgdiff}")
  
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)
  


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 3, 26)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 3, 26)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, 3, 26)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_8 (Flatten) │ (None, 78)        │          0 │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_9 (Flatten) │ (None, 78)        │          0 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_10          │ (None, 78)        │          0 │ input_layer_8[0]… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 32)        │      2,528 │ flatten_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 32)        │      2,528 │ flatten_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 32)        │      2,528 │ flatten_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 32)        │      1,056 │ dense_18[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 32)        │      1,056 │ dense_20[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 32)        │      1,056 │ dense_22[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 96)        │          0 │ dense_19[0][0],   │
│ (Concatenate)       │                   │            │ dense_21[0][0],   │
│                     │                   │            │ dense_23[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_11          │ (None, 96)        │          0 │ concatenate_2[0]… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 64)        │      6,208 │ flatten_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_25 (Dense)    │ (None, 64)        │      4,160 │ dense_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_26 (Dense)    │ (None, 1)         │         65 │ dense_25[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 21,185 (82.75 KB)

 Trainable params: 21,185 (82.75 KB)

 Non-trainable params: 0 (0.00 B)

Average value: 4.2942321762788795
Average diff: 1.3428647064103798
Average diff: 1.3028187858606088


In [11]:
history = model.fit(
    train_x,
    train_y,
    validation_data=(val_x, val_y),
    batch_size=batch_size,
    epochs=20,
    shuffle=True,
    callbacks=[tf.keras.callbacks.EarlyStopping(verbose=1, patience=20), tensorboard_callback]
)

model.save('model_sleep.h5')

Epoch 1/20
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.8551 - mae: 0.8551 - mse: 1.4829 - val_loss: 0.8137 - val_mae: 0.8138 - val_mse: 1.3071
Epoch 2/20
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.8514 - mae: 0.8514 - mse: 1.4698 - val_loss: 0.8328 - val_mae: 0.8329 - val_mse: 1.4162
Epoch 3/20
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.8501 - mae: 0.8501 - mse: 1.4668 - val_loss: 0.8158 - val_mae: 0.8159 - val_mse: 1.3250
Epoch 4/20
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.8458 - mae: 0.8458 - mse: 1.4514 - val_loss: 0.8175 - val_mae: 0.8177 - val_mse: 1.3425
Epoch 5/20
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.8431 - mae: 0.8431 - mse: 1.4439 - val_loss: 0.8156 - val_mae: 0.8158 - val_mse: 1.3342
Epoch 6/20
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.8415 - mae: 0.8415 - mse: 1.4387 - val_loss: 0.8096 - val_mae: 0.8098 - val_mse: 1.2767
Epoch 7/20
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.8390 - mae: 0.8390 - mse

In [12]:
predictions = []
for leaderboard_id, leaderboard_replays in test_data:
    try:
      pre, curr, post, score = preprocess_leaderboard_replays(leaderboard_replays)

      _predictions = model.predict([np.array(pre), np.array(curr), np.array(post)])

      real_sum = 0
      for prediction in score:
        real_sum += prediction

      real_avg = real_sum/_predictions.size
      real_percentage_score = 1 - clamp((real_avg / 1000) / 0.3, 0, 1)

      prediction_sum = 0
      for prediction in _predictions:
        prediction_sum += prediction[0]

      avg = prediction_sum/_predictions.size
      percentage_score = 1 - clamp((avg / 1000 ) / 0.3, 0, 1)

      predictions.append([f"https://beatleader.xyz/leaderboard/global/{leaderboard_id}", round(percentage_score, 5), round(real_percentage_score, 5), abs(round(real_percentage_score - percentage_score, 5))])
    except KeyboardInterrupt:
      raise
    except Exception as e:
      print(e)
      continue
with open('predictions.csv', 'w', newline='', encoding='utf-8') as f:
      writer = csv.writer(f)
      header = ["LeaderboardId", "Prediction", "Expected", "Difference"]
      writer.writerow(header)

      for prediction in predictions:
        writer.writerow(prediction)


math domain error
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
math domain error
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
math domain error
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
math domain error
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
math domain error
math domain error
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
23/23 ━━━━━━━━━━━━━━━━━━